The Recsys Chanllege 2015: \
https://2015.recsyschallenge.com/challenge.html\

In [1]:
# loading data
import pandas as pd

click = pd.read_csv('click_sep.csv', low_memory=False)

buy = pd.read_csv('buy_sep.csv',low_memory=False)

In [2]:
click.head()

,SessionID,TimeStamp,ItemID,Category
0,9293568,2014-09-01 18:07:00.855000+00:00,214853225,S
1,9293653,2014-09-01 10:38:47.087000+00:00,214834871,S
2,9293653,2014-09-01 10:39:49.115000+00:00,214849327,S
3,9293653,2014-09-01 10:40:31.736000+00:00,214828970,S
4,9293653,2014-09-01 10:41:01.640000+00:00,214849327,S


In [3]:
print('unique click session')
print(len(click.SessionID.unique()))

unique click session
99998


In [4]:
buy.head()

,SessionID,TimeStamp,ItemID,Price,Quantity
0,9641594,2014-09-01 09:09:25.575000+00:00,214853342,2093,2
1,9641594,2014-09-01 09:09:25.596000+00:00,214853340,837,2
2,9641594,2014-09-01 09:09:25.614000+00:00,214853420,1046,2
3,9431393,2014-09-01 13:38:48.351000+00:00,214846258,941,1
4,9431393,2014-09-01 13:38:48.414000+00:00,214853340,837,1


In [5]:
print('unique buy session')
print(len(buy.SessionID.unique()))

unique buy session
50000


In [6]:
# feature 1 
# In each click session, how many items were clicked                                                

feature1 = click.groupby('SessionID')['ItemID'].count().reset_index(name="item_cnt_in_ses")          

In [7]:
feature1.head()

,SessionID,item_cnt_in_ses
0,9194351,5
1,9194429,2
2,9194513,3
3,9194692,1
4,9194824,1


In [8]:
len(feature1)

99998

In [9]:
# feature 2 
# In which weekday the click activity happened.                                               

import datetime
click['click_weekday'] = pd.DatetimeIndex(click['TimeStamp']).dayofweek                         
click.head(5)

,SessionID,TimeStamp,ItemID,Category,click_weekday
0,9293568,2014-09-01 18:07:00.855000+00:00,214853225,S,0
1,9293653,2014-09-01 10:38:47.087000+00:00,214834871,S,0
2,9293653,2014-09-01 10:39:49.115000+00:00,214849327,S,0
3,9293653,2014-09-01 10:40:31.736000+00:00,214828970,S,0
4,9293653,2014-09-01 10:41:01.640000+00:00,214849327,S,0


In [10]:
# this step is to investigate duplicates 
feature2 = click[['SessionID','click_weekday']].drop_duplicates()
feature2.head()

,SessionID,click_weekday
0,9293568,0
1,9293653,0
6,9293662,0
8,9293704,0
10,9293797,0


In [11]:
len(feature2) 

100076

In [12]:
#feature3 
# The average price of all items clicked in each click session                           

# step 1 join certain columns from click and buy tables                                          
df = click.merge(buy[['ItemID','Price']], how = 'inner', on = 'ItemID').drop_duplicates()
print(df.head)
# this line might take some time to run, be patient.

# step 2 caculate average price from items clicked in each session
feature3 = df.groupby('SessionID')['Price'].mean().reset_index(name="ave_item_price")
print(feature3.head())

<bound method NDFrame.head of            SessionID                         TimeStamp     ItemID Category  \
0            9293568  2014-09-01 18:07:00.855000+00:00  214853225        S   
7            9293568  2014-09-01 18:07:00.855000+00:00  214853225        S   
22           9378348  2014-09-01 20:59:36.817000+00:00  214853225        S   
29           9378348  2014-09-01 20:59:36.817000+00:00  214853225        S   
44           9354957  2014-09-01 09:57:12.293000+00:00  214853225        S   
...              ...                               ...        ...      ...   
142184501   11292929  2014-09-27 11:38:11.837000+00:00  214554967        5   
142184502   11289399  2014-09-24 17:01:18.671000+00:00  214551294        8   
142184503   11299968  2014-09-27 18:41:26.992000+00:00  214769111       11   
142184504   11299968  2014-09-27 18:42:18.887000+00:00  214769589       11   
142184505   11298832  2014-09-27 17:28:44.687000+00:00  214835199        9   

           click_weekday  Price  

In [13]:
len(feature3)

96580

This feature will have missing values because the downsample caused some items are not in buy table anymore, therefore you won't find the price for the item.

In [14]:
# feature 4
# At what time period of a day the first click activity in a session happned (when a click session started).
# A feature from click dataset only.


# step 1: extract the hour from the timestamp
click['click_hourofday'] = pd.DatetimeIndex(click['TimeStamp']).hour 

# step 2: generate a new column to set the time of day into four categories, 0-6: 1, 7-11: 2, 12-17: 3, 18-24: 4.
click['click_timeofday'] = click['click_hourofday'].map (lambda x:1 if (x<6)
                                                                else 2 if (x<12)
                                                                else 3 if (x<18)
                                                                else 4 )
feature4 = click[['SessionID','click_timeofday']].drop_duplicates(subset='SessionID')
feature4.head()

,SessionID,click_timeofday
0,9293568,4
1,9293653,2
6,9293662,2
8,9293704,4
10,9293797,3


In [15]:
len(feature4)

99998

In [16]:
# feature 5
# whether a click activity happened close to the pay dates, which are usualy first day, 15th day or last day of a month.
# A feature from click dataset only.

# step 1: extract the day of month from the timestamp date.
click['click_dayofmonth'] = pd.DatetimeIndex(click['TimeStamp']).day                          

# step 2: set it into two categories, if the day is close to pay day (1th, 2nd, 3rd, 4th, 15th, 16th, 17th, 30th) set it to 1,
# otherwise, set it to 0.
click['click_payrollday'] = click['click_dayofmonth'].map (lambda x:1 if (x<5) or (x==30) or (x<18 & x>14) else 0 )

In [17]:
feature5 = click[['SessionID','click_payrollday']].drop_duplicates()
feature5.head()

,SessionID,click_payrollday
0,9293568,1
1,9293653,1
6,9293662,1
8,9293704,1
10,9293797,1


In [18]:
len(feature5)

100003

In [19]:
# feature 6
# the dwell time of each session in seconds.

# step 1: create a new dataframe having the time of the first click and the time of the last click of each session.
df6 = click.groupby('SessionID')['TimeStamp'].agg([min,max]).reset_index()
# this line may take some to run.

# step2: convert to timestamp
df6['begin']=pd.to_datetime(df6['min'])
df6['end']=pd.to_datetime(df6['max'])

# step 3: subtract to get the dwell time in seconds.
df6['dwell_time'] = (df6['end']-df6['begin']).astype('timedelta64[s]')
feature6 = df6[['SessionID','dwell_time']]
feature6.head()


,SessionID,dwell_time
0,9194351,320.0
1,9194429,60.0
2,9194513,883.0
3,9194692,0.0
4,9194824,0.0


In [20]:
len(feature6)

99998

In [21]:
# feature 7
# the number of popuplar items in each session.
# the popular items are the items sold more times than the average times of all sold items.

# step 1: if the sold time of an item is greater then average sold time, it is popular and set to 1, otherwise set to 0.
df7 = buy.groupby("ItemID")['Quantity'].sum().reset_index()
df7['pop_item'] = df7['Quantity'].map (lambda x:0 if (x<df7['Quantity'].mean()) else 1 )

# step 2: for each click session, get the number of popular items clicked.
df7_1 = click.merge(df7[['ItemID','pop_item']], how = 'inner', on = 'ItemID')
feature7 = df7_1.groupby('SessionID')['pop_item'].sum().reset_index(name='cnt_pop_item')
feature7.head()

,SessionID,cnt_pop_item
0,9194351,5
1,9194429,2
2,9194513,3
3,9194692,1
4,9194824,0


In [22]:
len(feature7)

96580

In [23]:
# feature 8
# the total score of cheap categories each session in click.
# the higher the score, the more items belong to expensive categories.

# step 1: get the average price of all items in each category
df8 = buy.merge(click[['ItemID','Category']], how = 'inner', on = 'ItemID').drop_duplicates()
# this line may take some time to run.

df8_1 = df8.groupby('Category')['Price'].mean().reset_index(name="ave_cat_price")
df8_1.sort_values(by='ave_cat_price')

,Category,ave_cat_price
7,2089437536,575.000000
14,8,682.918038
9,3,979.639705
3,11,1017.346000
6,2089300095,1151.000000
2,10,1898.272189
8,2089538467,1988.333333
15,9,2001.266529
0,0,2493.574872
16,S,2556.854473


In [24]:
# step 2: if the average price is below 1000, set the cheap_category score as 1, between 1000 and 2000: 2, 
# between 2000 and 3000: 3, greater than 3000: 4.
df8_1['cheap_category_score'] = df8_1['ave_cat_price'].map (lambda x:1 if (x < 1000) 
                                                            else 2 if (x<2000)
                                                            else 3 if (x<3000)
                                                            else 4)
# step 3: for each session of click, calculate the average cheap_category score for all items clicked.
df8_2 = click.merge(df8_1[['Category','cheap_category_score']], how = 'inner', on = 'Category')
feature8 = df8_2.groupby('SessionID')['cheap_category_score'].mean().reset_index(name='ave_cheap_cat_score')
feature8.tail()

,SessionID,ave_cheap_cat_score
99985,11561996,3.0
99986,11561999,3.0
99987,11562071,3.0
99988,11562104,4.0
99989,11562107,3.0


In [25]:
len(feature8)

99990

In [26]:
# feature 9
# the number of items in the most clicked categories for each session.

# step 1: find out the most clicked category. It is 'S'.
click['Category'].value_counts().head(1)


S    326603
Name: Category, dtype: int64

In [27]:
# step 2: for each click activity, if the category is "S", set to 1, otherwize set to 0.
click['is_most_clicked_cat'] = click['Category'].apply(lambda x: 1 if x == 'S' else 0)

# step 3: get the number of click activities which are in category 'S' in each session.
feature9 = click.groupby('SessionID')['is_most_clicked_cat'].sum().reset_index(name="cnt_most_click_cat")
feature9.head()

,SessionID,cnt_most_click_cat
0,9194351,5
1,9194429,2
2,9194513,3
3,9194692,1
4,9194824,0


In [28]:
len(feature9)

99998

In [29]:
# feature 10
# the number of mostly clicked items in each click session

# step 1: get the number of clicks on itemID and the 0.97 quantitle.
df10=click.groupby('ItemID')['SessionID'].count().reset_index()
df10.quantile(0.97)

ItemID       2.148596e+08
SessionID    1.690000e+02
Name: 0.97, dtype: float64

In [30]:
# step 2: the top 3 percent mostly clicked items are marked as most_click_item and set to 1, others are set to 0. 
df10['most_click_item'] = df10['SessionID'].map (lambda x:1 if (x > 169) else 0 )

# step 3: count the number of most_click_items clicked in each click session.
df10_1 = click.merge(df10[['ItemID','most_click_item']], how = 'left', on = 'ItemID').drop_duplicates()
feature10 = df10_1.groupby('SessionID')['most_click_item'].sum().reset_index(name='cnt_most_click_item')
feature10.head()

,SessionID,cnt_most_click_item
0,9194351,5
1,9194429,2
2,9194513,3
3,9194692,1
4,9194824,1


In [31]:
len(feature10)

99998

In [32]:
# feature 11
# the average time in seconds for an item in each click session

# step 1: concatenate feature1 which has click numbers and feature6 which has dwelling time.
df11 = pd.concat([feature1, feature6[['dwell_time']]], axis=1, sort=False)

# step 2: divide the dwelling time by number of clicks per session to get the click_rate. 
# higher the rate, less average time spend on each click activity.
# if only one item is clicked in a session, the rate is 0.
df11['click_rate']=df11['dwell_time']/df11['item_cnt_in_ses']
feature11 = df11[['SessionID','click_rate']]
feature11.head()

,SessionID,click_rate
0,9194351,64.000000
1,9194429,30.000000
2,9194513,294.333333
3,9194692,0.000000
4,9194824,0.000000


In [33]:
len(feature11)

99998

In [34]:
# feature 12
# the number of cheap items clicked in each click session

# step 1: get the distribution of prices of all items sold.
df12 = buy[['ItemID','Price']].drop_duplicates()
df12['Price'].describe()

count      8215.000000
mean       5164.282532
std       11277.619266
min          26.000000
25%         837.000000
50%        1884.000000
75%        4188.000000
max      208288.000000
Name: Price, dtype: float64

In [35]:
# step 2: for each item if it is cheaper than the price of 75% of all items, it is marked as a cheap item and set to 1, otherwise set to 0.
df12['cheap_item'] = df12['Price'].map (lambda x:1 if (x < 837) else 0 )
df12_1 = click.merge(df12[['ItemID','cheap_item']], how = 'inner', on = 'ItemID')

# step 3: count the number of cheap items clicked in each click session.
feature12 = df12_1.groupby('SessionID')['cheap_item'].sum().reset_index(name='cnt_cheap_item')
feature12.head()

,SessionID,cnt_cheap_item
0,9194351,1
1,9194429,0
2,9194513,2
3,9194692,0
4,9194824,0


In [36]:
len(feature12)

96580

In [37]:
# feature 13
# the number of items in most sold category clicked in each click session

# step 1: find out which is the most popular category (the most-sold category).
# then count the number of items that are belong to the most populat caterogy clicked in each click session. 
df13 = buy.groupby('ItemID')['Quantity'].sum().reset_index(name='total_sold_qty')
df13_1 = df13.merge(click[['ItemID','Category']],how='inner', on='ItemID').drop_duplicates()
df13_1.groupby('Category')['total_sold_qty'].sum().sort_values(ascending=False)

Category
S             131612
0              51119
3              50979
1              42162
8              13624
2              11127
5               8578
7               3370
6               2118
10              1773
4               1705
9               1033
11               502
12               203
2089300095         5
2089538467         5
2089437536         4
Name: total_sold_qty, dtype: int64

In [38]:
click['is_most_sold_cat'] = click['Category'].apply(lambda x: 1 if x == 'S' else 0)
feature13 = click.groupby('SessionID')['is_most_sold_cat'].sum().reset_index(name="most_sold_cat_num")
# because both the most clicked category (as in feature9) and most sold category are 'S', adding this feature won't help. So don't add this feature to the ABT.

In [39]:
# feature 14
# whether the first click item is a popular item (sold more than half of the items in buy)

# step 1: get the first click items.
df14 = click.sort_values(['SessionID','TimeStamp']).drop_duplicates(subset='SessionID')

In [40]:
# step 2: get the list of popular items (items sold more than half of the items in buy) (use some codes from feature7)
df7 = buy.groupby("ItemID")['Quantity'].sum().reset_index()
df7['pop_item'] = df7['Quantity'].map (lambda x:0 if (x<df7['Quantity'].mean()) else 1 )
pop_sold_item = df7.loc[df7['pop_item']==1].ItemID.unique()

In [41]:
# step 3: to chech whether the first click item is a popular item.
df14['first_click_pop_item'] = df14['ItemID'].apply(lambda x: 1 if x in pop_sold_item else 0)

In [42]:
feature14 = df14[['SessionID','first_click_pop_item']]
feature14.tail(10)

,SessionID,first_click_pop_item
435417,11561877,1
435411,11561938,1
435415,11561974,0
435262,11561989,1
435264,11561994,1
435267,11561996,1
435266,11561999,1
435274,11562071,1
435269,11562104,1
435270,11562107,0


In [43]:
len(feature14)

99998

In [44]:
# feature 15
# whether the last click item is a popular item (sold more than half of the items in buy)

# step 1: get the first click items.
df15 = click.sort_values(['SessionID','TimeStamp'],ascending=[True, False]).drop_duplicates(subset='SessionID')

In [45]:
# step 2: to chech whether the first click item is a popular item (similar as for feature 14).
df15['last_click_pop_item'] = df15['ItemID'].apply(lambda x: 1 if x in pop_sold_item else 0)

In [46]:
feature15 = df15[['SessionID','last_click_pop_item']]
feature15.tail(10)

,SessionID,last_click_pop_item
435418,11561877,1
435414,11561938,1
435416,11561974,1
435263,11561989,1
435265,11561994,0
435268,11561996,1
435266,11561999,1
435278,11562071,1
435269,11562104,1
435273,11562107,1


In [47]:
len(feature15)

99998

### Final Step 
Create ABT on session level

If there is a record in buy session, then it means the buy activitiy happened and that buy session should be marked as 1.
In click sessions, there are many session ids not in the buy table, that means the click activities happened but buy activity did not happen, then these click sessions should be marked as 0.

In [48]:
buy['buy_or_not'] = 1
click_ABT = click[['SessionID']]
ABT = click_ABT.merge(buy[['SessionID','buy_or_not']], how = 'left', on = 'SessionID').drop_duplicates().fillna(0)
ABT.tail(10)

,SessionID,buy_or_not
1448127,11299607,0.0
1448129,11299701,0.0
1448130,11299683,1.0
1448134,11299738,1.0
1448148,11299742,0.0
1448149,11299723,0.0
1448155,11299767,0.0
1448159,11299803,0.0
1448160,11299794,1.0
1448180,11299793,1.0


In [49]:
len(ABT)

99998

### Join all features together with predictive target column in the final ABT ---- 10 Points

In [50]:
ABT2=pd.merge(feature1, feature2, how = 'inner', on = 'SessionID')
ABT3=pd.merge(ABT2, feature3, how = 'left', on = 'SessionID')
ABT4=pd.merge(ABT3, feature4, how = 'inner', on = 'SessionID')
ABT5=pd.merge(ABT4, feature5, how = 'inner', on = 'SessionID')
ABT6=pd.merge(ABT5, feature6, how = 'inner', on = 'SessionID')
ABT7=pd.merge(ABT6, feature7, how = 'left', on = 'SessionID')
ABT8=pd.merge(ABT7, feature8, how = 'left', on = 'SessionID')
ABT9=pd.merge(ABT8, feature9, how = 'inner', on = 'SessionID')
ABT10=pd.merge(ABT9, feature10, how = 'inner', on = 'SessionID')
ABT11=pd.merge(ABT10, feature11, how = 'inner', on = 'SessionID')
ABT12=pd.merge(ABT11, feature12, how = 'left', on = 'SessionID')
ABT14=pd.merge(ABT12, feature14, how = 'inner', on = 'SessionID')
ABT15=pd.merge(ABT14, feature15, how = 'inner', on = 'SessionID')
ABT_final=pd.merge(ABT15, ABT, how = 'inner', on = 'SessionID')

In [51]:
ABT_final.head(10)

,SessionID,item_cnt_in_ses,click_weekday,ave_item_price,click_timeofday,click_payrollday,dwell_time,cnt_pop_item,ave_cheap_cat_score,cnt_most_click_cat,cnt_most_click_item,click_rate,cnt_cheap_item,first_click_pop_item,last_click_pop_item,buy_or_not
0,9194351,5,0,1534.777778,2,1,320.0,5.0,3.000000,5,5,64.000000,1.0,1,1,0.0
1,9194429,2,0,1412.500000,2,1,60.0,2.0,3.000000,2,2,30.000000,0.0,1,1,0.0
2,9194513,3,0,1470.500000,2,1,883.0,3.0,3.000000,3,3,294.333333,2.0,1,1,0.0
3,9194692,1,0,3664.000000,4,1,0.0,1.0,3.000000,1,1,0.000000,0.0,1,1,0.0
4,9194824,1,0,9424.000000,1,1,0.0,0.0,3.000000,0,1,0.000000,0.0,0,0,0.0
5,9195173,4,0,2219.200000,3,1,95.0,4.0,3.000000,1,1,23.750000,0.0,1,1,0.0
6,9195209,3,0,16650.000000,3,1,2099.0,0.0,3.000000,0,0,699.666667,0.0,0,0,1.0
7,9195293,6,0,719.222222,1,1,1141.0,6.0,3.000000,3,6,190.166667,5.0,1,1,1.0
8,9195417,8,0,941.400000,3,1,2090.0,8.0,4.000000,0,8,261.250000,2.0,1,1,0.0
9,9195426,9,0,8388.111111,3,1,1318.0,3.0,3.111111,4,0,146.444444,0.0,0,0,0.0


In [52]:
ABT_final.to_csv('Recsys_2015.csv')